In [1]:
!pip install -U transformers trl peft datasets evaluate rouge_score underthesea bitsandbytes thefuzz sentence-transformers numpy dotenv
# Tải tài nguyên NLTK cho METEOR
import nltk
nltk.download('wordnet')
nltk.download('omw-1.4')

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.4/10.4 MB 40.9 MB/s eta 0:00:0000:010:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 336.4/336.4 KB 56.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 411.1/411.1 KB 57.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 KB 49.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 KB 36.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.9/20.9 MB 59.2 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 26.3 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 345.7/345.7 KB 87.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 67.5 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.5/78.5 KB 31.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 781.7/781.7 KB 99.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


True

In [2]:
# from google.colab import drive
# drive.mount('/content/drive')

In [3]:
import os
os.environ["WANDB_DISABLED"] = "true"
import json
import pandas as pd
from datasets import Dataset
from sklearn.model_selection import train_test_split
import string
from underthesea import word_tokenize
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments, EarlyStoppingCallback
from peft import LoraConfig, get_peft_model
from trl import SFTTrainer
import evaluate

/root/model/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


In [4]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from dotenv import load_dotenv
import os
import torch
from peft import LoraConfig, get_peft_model

# Tải biến môi trường từ file .env
load_dotenv()

# Đọc access token từ biến môi trường
hf_token = os.getenv("HF_DAMA2")
if not hf_token:
    raise ValueError("Không tìm thấy HF_DAMA2 trong file .env. Vui lòng thêm token vào file .env với định dạng: HF_DAMA2=your_token")

def load_model_and_tokenizer(quantization="int8"):  # Thêm tham số quantization
    model_name = "vietgpt/dama-2-7b-chat"

    # Cấu hình quantization với bitsandbytes
    if quantization == "int8":
        quantization_config = BitsAndBytesConfig(
            load_in_8bit=True,  # Quantize thành INT8
            bnb_8bit_compute_dtype=torch.bfloat16,  # Dùng bfloat16 để tính toán
            bnb_8bit_use_double_quant=True,  # Double quantization để tăng hiệu quả
        )
    elif quantization == "int4":
        quantization_config = BitsAndBytesConfig(
            load_in_4bit=True,  # Quantize thành INT4
            bnb_4bit_compute_dtype=torch.bfloat16,  # Dùng bfloat16 để tính toán
            bnb_4bit_use_double_quant=True,  # Double quantization
            bnb_4bit_quant_type="nf4",  # Dùng NF4 (Normalized Float 4-bit) để tối ưu
        )
    else:
        quantization_config = None  # Không quantize

    # Tải tokenizer và mô hình với token xác thực
    tokenizer = AutoTokenizer.from_pretrained(
        model_name,
        token=hf_token  # Sử dụng token từ biến môi trường
    )
    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        quantization_config=quantization_config,
        torch_dtype=torch.bfloat16,
        device_map="auto",
        token=hf_token  # Sử dụng token từ biến môi trường
    )
    print(model)
    # Cấu hình LoRA
    peft_config = LoraConfig(
        r=32,
        lora_alpha=32,
        lora_dropout=0.5,
        bias="none",
        task_type="CAUSAL_LM",
        target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"]  # Thử thay đổi ở đây
    )
    model = get_peft_model(model, peft_config)
    return model, tokenizer, peft_config

# Tải mô hình với INT8 quantization
model, tokenizer, peft_config = load_model_and_tokenizer(quantization="int8")

Loading checkpoint shards: 100%|██████████| 2/2 [00:21<00:00, 10.88s/it]


LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(50261, 4096, padding_idx=0)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear8bitLt(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear8bitLt(in_features=4096, out_features=4096, bias=False)
          (v_proj): Linear8bitLt(in_features=4096, out_features=4096, bias=False)
          (o_proj): Linear8bitLt(in_features=4096, out_features=4096, bias=False)
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear8bitLt(in_features=4096, out_features=11008, bias=False)
          (up_proj): Linear8bitLt(in_features=4096, out_features=11008, bias=False)
          (down_proj): Linear8bitLt(in_features=11008, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
      )
    )
 

In [5]:
# Định nghĩa formatting_func với apply_chat_template
def formatting_func(example):
    if not all(k in example for k in ['input', 'output']):
        print('Thiếu key trong example:', example)
        return ''
    messages = [
        {"role": "user", "content": example['input']},
        {"role": "assistant", "content": example['output']}
    ]
    return tokenizer.apply_chat_template(messages, tokenize=False)

In [6]:
import os
import json
import pandas as pd
from datasets import Dataset
from sklearn.model_selection import train_test_split
import string
from underthesea import word_tokenize
from thefuzz import fuzz

def preprocess_text(text):
    text = text.lower()
    text = text.translate(str.maketrans("", "", string.punctuation))
    tokens = word_tokenize(text)
    text = " ".join(tokens)
    return text

def extract_json_from_folder(folder_path):
    dataset = []
    for filename in os.listdir(folder_path):
        if filename.endswith(".json"):
            file_path = os.path.join(folder_path, filename)
            try:
                with open(file_path, 'r', encoding='utf-8') as f:
                    json_data = json.load(f)
                    for item in json_data:
                        if all(k in item for k in ['input', 'output']):
                            item['input'] = preprocess_text(item['input'])
                            item['output'] = preprocess_text(item['output'])
                            dataset.append(item)
                        else:
                            print(f"Thiếu trường trong {filename}: {item}")
            except json.JSONDecodeError:
                print(f"Không thể parse JSON từ {filename}")
            except Exception as e:
                print(f"Lỗi khi đọc {filename}: {e}")
    return dataset

folder_path = "/root/model/data"
dataset = extract_json_from_folder(folder_path)

df = pd.DataFrame(dataset)
print("Số lượng giá trị duy nhất trong 'input' (exact):", df['input'].nunique())
print("Số lượng giá trị duy nhất trong 'output' (exact):", df['output'].nunique())
print("Tổng số hàng:", len(df))

# Fuzzy matching để tìm các record tương tự
similarity_threshold = 90  # Ngưỡng độ tương đồng (90%)
input_pairs = []
output_pairs = []

# Tìm các cặp input và output tương tự
for i in range(len(df)):
    for j in range(i + 1, len(df)):
        input_sim = fuzz.ratio(df['input'].iloc[i], df['input'].iloc[j])
        if input_sim >= similarity_threshold:
            input_pairs.append((i, j, input_sim))
        output_sim = fuzz.ratio(df['output'].iloc[i], df['output'].iloc[j])
        if output_sim >= similarity_threshold:
            output_pairs.append((i, j, output_sim))

print(f"Số cặp input tương tự (>{similarity_threshold}%):", len(input_pairs))
print(f"Số cặp output tương tự (>{similarity_threshold}%):", len(output_pairs))

# Loại bỏ các record có input hoặc output tương tự (giữ record đầu tiên)
indices_to_keep = set(range(len(df)))
for i, j, _ in input_pairs:
    if j in indices_to_keep:
        indices_to_keep.remove(j)
for i, j, _ in output_pairs:
    if j in indices_to_keep:
        indices_to_keep.remove(j)

df = df.iloc[list(indices_to_keep)].reset_index(drop=True)
print("Số hàng sau khi xóa record tương tự:", len(df))

# Kiểm tra lại độ unique
print("Số lượng giá trị duy nhất trong 'input' (sau xử lý):", df['input'].nunique())
print("Số lượng giá trị duy nhất trong 'output' (sau xử lý):", df['output'].nunique())

# Chia train/validation
full_dataset = Dataset.from_pandas(df[['input', 'output']])
train_df, eval_df = train_test_split(df, test_size=0.2, random_state=42)
train_dataset = Dataset.from_pandas(train_df[['input', 'output']])
eval_dataset = Dataset.from_pandas(eval_df[['input', 'output']])
print(train_df[['input', 'output']].head())

Số lượng giá trị duy nhất trong 'input' (exact): 1599
Số lượng giá trị duy nhất trong 'output' (exact): 1659
Tổng số hàng: 1700
Số cặp input tương tự (>90%): 320
Số cặp output tương tự (>90%): 41
Số hàng sau khi xóa record tương tự: 1472
Số lượng giá trị duy nhất trong 'input' (sau xử lý): 1472
Số lượng giá trị duy nhất trong 'output' (sau xử lý): 1472
                                                  input  \
998   tự kỷ có phải do dùng thiết bị điện tử nhiều k...   
254                     trẻ nói nhại lại lời người khác   
1074              tự kỷ có nên dùng thuốc an thần không   
643    trẻ không biết tự khen ngợi bản thân khi làm tốt   
1451                    trẻ 2 tuổi chỉ hét mà không nói   

                                                 output  
998   hiện chưa có bằng chứng khoa học cho thấy thiế...  
254   việc trẻ lặp lại lời nói người khác thường xuy...  
1074  không dùng thuốc an thần bừa bãi cho trẻ tự kỷ...  
643   không ghi nhận thành công khiến trẻ thiếu tự t...  


In [7]:
import evaluate
import numpy as np
from sentence_transformers import SentenceTransformer, util
def evaluate_metrics(predictions, references):
    # Tải các độ đo
    rouge = evaluate.load("rouge")
    meteor = evaluate.load("meteor")

    # Tính các độ đo
    rouge_results = rouge.compute(predictions=predictions, references=references)
    meteor_results = meteor.compute(predictions=predictions, references=references)

    # Tải mô hình nhúng câu để tính Cosine Similarity
    embedder = SentenceTransformer('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2')

    # Tính embeddings cho dự đoán và tham chiếu
    pred_embeddings = embedder.encode(predictions, convert_to_tensor=True)
    ref_embeddings = embedder.encode(references, convert_to_tensor=True)

    # Tính Cosine Similarity giữa từng cặp dự đoán-tham chiếu
    cosine_scores = util.cos_sim(pred_embeddings, ref_embeddings)
    # Lấy trung bình Cosine Similarity (chỉ lấy đường chéo chính, vì mỗi dự đoán chỉ so với tham chiếu tương ứng)
    avg_cosine_similarity = np.mean([cosine_scores[i][i].item() for i in range(len(predictions))])
    
    # Gộp kết quả
    metrics = {
        "rouge1": rouge_results["rouge1"],
        "rouge2": rouge_results["rouge2"],
        "rougeL": rouge_results["rougeL"],
        "meteor": meteor_results["meteor"],
        "cosine_similarity": avg_cosine_similarity,
    }
    return metrics

def generate_predictions(model, tokenizer, inputs, max_length=200):
    """Tạo dự đoán từ mô hình cho các đầu vào."""
    model.eval()
    predictions = []
    # Đảm bảo pad_token được thiết lập
    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token

    for input_text in inputs:
        # Tiền xử lý input để đồng bộ với huấn luyện
        input_text = preprocess_text(input_text)
        prompt = f"<s>[INST] {input_text} [/INST]"
        inputs_encoded = tokenizer(prompt, return_tensors="pt", padding=True, truncation=True).to(model.device)
        with torch.no_grad():
            outputs = model.generate(
                **inputs_encoded,
                max_length=max_length,
                num_return_sequences=1,
                pad_token_id=tokenizer.pad_token_id
            )
        generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
        pred = generated_text.split("[/INST]")[-1].strip()
        # Tiền xử lý dự đoán để đồng bộ với tham chiếu
        pred = preprocess_text(pred)
        predictions.append(pred)
    return predictions

In [8]:
import json
import time
from transformers import TrainingArguments
from trl import SFTTrainer
from transformers import EarlyStoppingCallback

# Loại bỏ cột không cần thiết để tránh cảnh báo
train_dataset = train_dataset.remove_columns(['__index_level_0__'] if '__index_level_0__' in train_dataset.column_names else [])
eval_dataset = eval_dataset.remove_columns(['__index_level_0__'] if '__index_level_0__' in eval_dataset.column_names else [])

model_name = "vietgpt/dama-2-7b-chat"

# Đo thời gian chạy đơn
print("\n=== Bắt đầu chạy đơn ===")
start_time = time.time()

# Cấu hình huấn luyện cho Single Run
training_arguments_single = TrainingArguments(
    output_dir=f"./results_single_{model_name.split('/')[-1]}",
    num_train_epochs=3,
    per_device_train_batch_size=1,
    gradient_accumulation_steps=16,
    optim="paged_adamw_32bit",
    save_steps=100,
    logging_steps=10,
    learning_rate=5e-5,
    weight_decay=0.1,
    fp16=False,
    bf16=True,
    max_grad_norm=0.3,
    warmup_ratio=0.1,
    group_by_length=True,
    lr_scheduler_type="cosine",
    eval_strategy="steps",
    eval_steps=10,
    logging_strategy="steps",
    log_level="info",
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    greater_is_better=False,
)

# Huấn luyện Single Run
trainer_single = SFTTrainer(
    model=model,
    args=training_arguments_single,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    peft_config=peft_config,
    formatting_func=formatting_func,
    callbacks=[
        EarlyStoppingCallback(
            early_stopping_patience=3,
            early_stopping_threshold=0.01,
        )
    ],
)
print(f"Train dataset size: {len(train_dataset)}, Eval dataset size: {len(eval_dataset)}")
trainer_single.train()

# Lưu mô hình Single Run
model.save_pretrained(f"./finetuned_{model_name.split('/')[-1]}_single")
tokenizer.save_pretrained(f"./finetuned_{model_name.split('/')[-1]}_single")

# Đánh giá Single Run
test_inputs = eval_df['input'].tolist()
test_references = eval_df['output'].tolist()
predictions_single = generate_predictions(model, tokenizer, test_inputs)
metrics_single = evaluate_metrics(predictions_single, test_references)
print(f"Single Run Metrics for {model_name}:", metrics_single)

# Lưu metrics vào file
metrics_file = f"single_run_metrics_{model_name.split('/')[-1]}.json"
try:
    with open(metrics_file, "w", encoding="utf-8") as f:
        json.dump(metrics_single, f, indent=4, ensure_ascii=False)
    print(f"Đã lưu metrics chạy đơn vào: {metrics_file}")
except Exception as e:
    print(f"Lỗi khi lưu metrics chạy đơn: {e}")

# In thời gian chạy
end_time = time.time()
duration = end_time - start_time
print(f"Thời gian chạy đơn: {duration:.2f} giây")

# Kiểm tra mẫu dự đoán
for i in range(5):
    print(f"Input: {test_inputs[i]}")
    print(f"Prediction: {predictions_single[i]}")
    print(f"Reference: {test_references[i]}\n")

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).



=== Bắt đầu chạy đơn ===


Truncating eval dataset: 100%|██████████| 295/295 [00:00<00:00, 115110.21 examples/s]
Using auto half precision backend
No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.
The following columns in the training set don't have a corresponding argument in `PeftModelForCausalLM.forward` and have been ignored: input, output, text. If input, output, text are not expected by `PeftModelForCausalLM.forward`,  you can safely ignore this message.
***** Running training *****
  Num examples = 1,177
  Num Epochs = 3
  Instantaneous batch size per device = 1
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 16
  Total optimization steps = 219
  Number of trainable parameters = 79,953,920


Train dataset size: 1177, Eval dataset size: 295


/root/model/venv/lib/python3.10/site-packages/bitsandbytes/autograd/_functions.py:315: UserWarning: MatMul8bitLt: inputs will be cast from torch.bfloat16 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")


Step,Training Loss,Validation Loss
10,4.374400,4.475065
20,4.061000,3.462521
30,3.039500,2.622163
40,2.319200,2.050639
50,1.962600,1.875167
60,1.890300,1.809317
70,1.725800,1.746339
80,1.685000,1.713642
90,1.644300,1.684539
100,1.638000,1.663903


The following columns in the evaluation set don't have a corresponding argument in `PeftModelForCausalLM.forward` and have been ignored: input, output, text. If input, output, text are not expected by `PeftModelForCausalLM.forward`,  you can safely ignore this message.

***** Running Evaluation *****
  Num examples = 295
  Batch size = 8
The following columns in the evaluation set don't have a corresponding argument in `PeftModelForCausalLM.forward` and have been ignored: input, output, text. If input, output, text are not expected by `PeftModelForCausalLM.forward`,  you can safely ignore this message.

***** Running Evaluation *****
  Num examples = 295
  Batch size = 8
The following columns in the evaluation set don't have a corresponding argument in `PeftModelForCausalLM.forward` and have been ignored: input, output, text. If input, output, text are not expected by `PeftModelForCausalLM.forward`,  you can safely ignore this message.

***** Running Evaluation *****
  Num examples = 2

Single Run Metrics for vietgpt/dama-2-7b-chat: {'rouge1': np.float64(0.615354014833372), 'rouge2': np.float64(0.279524855165236), 'rougeL': np.float64(0.3940296014840703), 'meteor': np.float64(0.32146654101083755), 'cosine_similarity': np.float64(0.7181124540708833)}
Đã lưu metrics chạy đơn vào: single_run_metrics_dama-2-7b-chat.json
Thời gian chạy đơn: 2460.00 giây
Input: trẻ không biết sử dụng hình ảnh minh họa kết quả hoạt động
Prediction: trẻ không biết sử dụng hình ảnh minh họa kết quả hoạt động không dùng hình ảnh minh họa là thiếu kỹ năng trình bày – nên luyện qua tranh ảnh hoặc video ngắn
Reference: không chụp hình hoặc chọn hình mờ nhạt – nên luyện chọn hình rõ người – rõ hành động – có cảm xúc

Input: trẻ không dùng tay để chỉ vật
Prediction: trẻ không dùng tay để chỉ vật không chỉ tay là thiếu kỹ năng chỉ – đây là dấu hiệu chậm phát triển ngôn ngữ
Reference: việc trẻ không dùng tay để chỉ khi muốn thể hiện nhu cầu là dấu hiệu quan trọng để nhận biết trẻ có khó khăn trong gia

In [10]:
from sklearn.model_selection import KFold
import numpy as np
import gc
import time
import torch

# Cấu hình KFold
kf = KFold(n_splits=5, shuffle=True, random_state=42)
fold_metrics = []
fold_models = []
fold_times = []
total_start_time = time.time()

# Lặp qua từng fold
for fold, (train_idx, eval_idx) in enumerate(kf.split(df)):
    print(f"\nTraining Fold {fold + 1}...")
    fold_start_time = time.time()

    # Tạo tập train và eval cho fold hiện tại
    train_fold = df.iloc[train_idx][['input', 'output']]
    eval_fold = df.iloc[eval_idx][['input', 'output']]
    train_fold_dataset = Dataset.from_pandas(train_fold)
    eval_fold_dataset = Dataset.from_pandas(eval_fold)

    # Loại bỏ cột không cần thiết
    train_fold_dataset = train_fold_dataset.remove_columns(['__index_level_0__'] if '__index_level_0__' in train_fold_dataset.column_names else [])
    eval_fold_dataset = eval_fold_dataset.remove_columns(['__index_level_0__'] if '__index_level_0__' in eval_fold_dataset.column_names else [])

    # Tải lại mô hình gốc với INT4 quantization
    model, tokenizer, peft_config = load_model_and_tokenizer(quantization="int4")
    print(f"Fold {fold + 1} - Train size: {len(train_fold_dataset)}, Eval size: {len(eval_fold_dataset)}")

    # Cấu hình huấn luyện cho fold
    training_arguments_fold = TrainingArguments(
        output_dir=f"./results_{model_name.split('/')[-1]}fold_{fold + 1}",
        num_train_epochs=3,
        per_device_train_batch_size=1,
        gradient_accumulation_steps=16,
        optim="paged_adamw_32bit",
        save_steps=100,
        logging_steps=10,
        learning_rate=5e-5,
        weight_decay=0.1,
        fp16=False,
        bf16=True,
        max_grad_norm=0.3,
        warmup_ratio=0.1,
        group_by_length=True,
        lr_scheduler_type="cosine",
        eval_strategy="steps",
        eval_steps=10,
        logging_strategy="steps",
        log_level="info",
        load_best_model_at_end=True,
        metric_for_best_model="eval_loss",
        greater_is_better=False,
    )

    # Huấn luyện fold
    trainer_fold = SFTTrainer(
        model=model,
        args=training_arguments_fold,
        train_dataset=train_fold_dataset,
        eval_dataset=eval_fold_dataset,
        peft_config=peft_config,
        formatting_func=formatting_func,
        callbacks=[
            EarlyStoppingCallback(
                early_stopping_patience=3,
                early_stopping_threshold=0.01,
            )
        ],
    )
    trainer_fold.train()

    # Lưu mô hình fold
    fold_path = f"./finetuned_vistral_fold_{fold + 1}"
#pragma warning disable format
    model.save_pretrained(fold_path)
    tokenizer.save_pretrained(fold_path)
    fold_models.append(fold_path)

    # Đánh giá fold
    test_inputs_fold = eval_fold['input'].tolist()
    test_references_fold = eval_fold['output'].tolist()
    predictions_fold = generate_predictions(model, tokenizer, test_inputs_fold)
    metrics_fold = evaluate_metrics(predictions_fold, test_references_fold)
    print(f"Fold {fold + 1} Metrics:", metrics_fold)
    fold_metrics.append(metrics_fold)

    # Lưu metrics của fold
    with open(f"fold_{fold + 1}_metrics_vistral.json", "w") as f:
        json.dump(metrics_fold, f, indent=4)

    # Đo thời gian chạy của fold
    fold_end_time = time.time()
    fold_duration = fold_end_time - fold_start_time
    fold_times.append(fold_duration)
    print(f"Thời gian chạy Fold {fold + 1}: {fold_duration:.2f} giây")

    # Dọn dẹp bộ nhớ
    del model, trainer_fold
    torch.cuda.empty_cache()
    gc.collect()

# Tính tổng thời gian và thời gian trung bình
total_end_time = time.time()
total_duration = total_end_time - total_start_time
print(f"\n=== Kết thúc huấn luyện K-Fold ===")
print(f"Tổng thời gian chạy: {total_duration:.2f} giây")
print(f"Thời gian trung bình mỗi fold: {np.mean(fold_times):.2f} giây")

# Tính trung bình metrics qua các fold
avg_metrics = {
    "rouge1": np.mean([m["rouge1"] for m in fold_metrics]),
    "rouge2": np.mean([m["rouge2"] for m in fold_metrics]),
    "rougeL": np.mean([m["rougeL"] for m in fold_metrics]),
    "meteor": np.mean([m["meteor"] for m in fold_metrics]),
    "cosine_similarity": np.mean([m["cosine_similarity"] for m in fold_metrics]),
}
print("\nAverage Cross-Validation Metrics:", avg_metrics)

# Lưu metrics trung bình
with open("cross_validation_metrics.json", "w") as f:
    json.dump(avg_metrics, f, indent=4)


Training Fold 1...


loading file vocab.json from cache at /root/.cache/huggingface/hub/models--vietgpt--dama-2-7b-chat/snapshots/efacdb053b58e26fe1de9e6b6faf0224367f78a1/vocab.json
loading file merges.txt from cache at /root/.cache/huggingface/hub/models--vietgpt--dama-2-7b-chat/snapshots/efacdb053b58e26fe1de9e6b6faf0224367f78a1/merges.txt
loading file tokenizer.json from cache at /root/.cache/huggingface/hub/models--vietgpt--dama-2-7b-chat/snapshots/efacdb053b58e26fe1de9e6b6faf0224367f78a1/tokenizer.json
loading file added_tokens.json from cache at /root/.cache/huggingface/hub/models--vietgpt--dama-2-7b-chat/snapshots/efacdb053b58e26fe1de9e6b6faf0224367f78a1/added_tokens.json
loading file special_tokens_map.json from cache at /root/.cache/huggingface/hub/models--vietgpt--dama-2-7b-chat/snapshots/efacdb053b58e26fe1de9e6b6faf0224367f78a1/special_tokens_map.json
loading file tokenizer_config.json from cache at /root/.cache/huggingface/hub/models--vietgpt--dama-2-7b-chat/snapshots/efacdb053b58e26fe1de9e6b6fa

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(50261, 4096, padding_idx=0)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=11008, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=11008, bias=False)
          (down_proj): Linear4bit(in_features=11008, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
      )
    )
    (norm): Lla

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
PyTorch: setting up devices


Fold 1 - Train size: 1177, Eval size: 295


...

Fold 5 - Train size: 1178, Eval size: 294


loading file vocab.json from cache at /root/.cache/huggingface/hub/models--vietgpt--dama-2-7b-chat/snapshots/efacdb053b58e26fe1de9e6b6faf0224367f78a1/vocab.json
loading file merges.txt from cache at /root/.cache/huggingface/hub/models--vietgpt--dama-2-7b-chat/snapshots/efacdb053b58e26fe1de9e6b6faf0224367f78a1/merges.txt
loading file tokenizer.json from cache at /root/.cache/huggingface/hub/models--vietgpt--dama-2-7b-chat/snapshots/efacdb053b58e26fe1de9e6b6faf0224367f78a1/tokenizer.json
loading file added_tokens.json from cache at /root/.cache/huggingface/hub/models--vietgpt--dama-2-7b-chat/snapshots/efacdb053b58e26fe1de9e6b6faf0224367f78a1/added_tokens.json
loading file special_tokens_map.json from cache at /root/.cache/huggingface/hub/models--vietgpt--dama-2-7b-chat/snapshots/efacdb053b58e26fe1de9e6b6faf0224367f78a1/special_tokens_map.json
loading file tokenizer_config.json from cache at /root/.cache/huggingface/hub/models--vietgpt--dama-2-7b-chat/snapshots/efacdb053b58e26fe1de9e6b6fa

Step,Training Loss,Validation Loss
10,3.674500,3.779777
20,3.481100,3.175684
30,2.705400,2.360362
40,2.168600,2.058716
50,1.954200,1.908956
60,1.816400,1.846899
70,1.734700,1.782158
80,1.737400,1.737031
90,1.610600,1.712836
100,1.577400,1.685128


The following columns in the evaluation set don't have a corresponding argument in `PeftModelForCausalLM.forward` and have been ignored: input, output, text. If input, output, text are not expected by `PeftModelForCausalLM.forward`,  you can safely ignore this message.

***** Running Evaluation *****
  Num examples = 294
  Batch size = 8
The following columns in the evaluation set don't have a corresponding argument in `PeftModelForCausalLM.forward` and have been ignored: input, output, text. If input, output, text are not expected by `PeftModelForCausalLM.forward`,  you can safely ignore this message.

***** Running Evaluation *****
  Num examples = 294
  Batch size = 8
The following columns in the evaluation set don't have a corresponding argument in `PeftModelForCausalLM.forward` and have been ignored: input, output, text. If input, output, text are not expected by `PeftModelForCausalLM.forward`,  you can safely ignore this message.

***** Running Evaluation *****
  Num examples = 2

Fold 5 Metrics: {'rouge1': np.float64(0.6186252424333942), 'rouge2': np.float64(0.2930516278453371), 'rougeL': np.float64(0.3991195690132021), 'meteor': np.float64(0.3510191560313527), 'cosine_similarity': np.float64(0.7293402220116181)}
Thời gian chạy Fold 5: 2067.52 giây

=== Kết thúc huấn luyện K-Fold ===
Tổng thời gian chạy: 9721.49 giây
Thời gian trung bình mỗi fold: 1943.59 giây

Average Cross-Validation Metrics: {'rouge1': np.float64(0.6106109665779863), 'rouge2': np.float64(0.2833767321482045), 'rougeL': np.float64(0.39559231029704495), 'meteor': np.float64(0.3327652324810843), 'cosine_similarity': np.float64(0.7220307202843541)}


In [ ]:
from peft import PeftModel
from transformers import AutoModelForCausalLM, BitsAndBytesConfig
import copy

# Tải mô hình gốc với INT8 quantization
base_model, tokenizer, _ = load_model_and_tokenizer(quantization="int8")

# Kiểm tra xem có mô hình fold nào để gộp không
if not fold_models:
    raise ValueError("No fold models found to merge.")

# Thu thập và trung bình trọng số LoRA từ các fold
state_dicts = []
for fold_path in fold_models:
    fold_model = PeftModel.from_pretrained(base_model, fold_path, is_trainable=False)
    state_dicts.append(copy.deepcopy(fold_model.state_dict()))
    del fold_model
    torch.cuda.empty_cache()
    gc.collect()

# Trung bình trọng số
averaged_state_dict = state_dicts[0]
for key in averaged_state_dict:
    for i in range(1, len(state_dicts)):
        averaged_state_dict[key] += state_dicts[i][key]
    averaged_state_dict[key] /= len(state_dicts)

# Tải trọng số trung bình vào mô hình gốc
base_model.load_state_dict(averaged_state_dict, strict=False)

# Lưu mô hình gộp
merged_model_path = "./finetuned_dama_2_merged"
base_model.save_pretrained(merged_model_path)
tokenizer.save_pretrained(merged_model_path)

# Dọn dẹp bộ nhớ
del base_model
torch.cuda.empty_cache()
gc.collect()

# Tải mô hình gộp để đánh giá với INT8 quantization
merged_model = AutoModelForCausalLM.from_pretrained(
    merged_model_path,
    quantization_config=BitsAndBytesConfig(
        load_in_8bit=True,
        bnb_8bit_compute_dtype=torch.bfloat16,
        bnb_8bit_use_double_quant=True
    ),
    torch_dtype=torch.bfloat16,
    device_map="auto"
)

# Đánh giá mô hình gộp trên tập validation
test_inputs = eval_df['input'].tolist()
test_references = eval_df['output'].tolist()
predictions_merged = generate_predictions(merged_model, tokenizer, test_inputs)
metrics_merged = evaluate_metrics(predictions_merged, test_references)
print("Merged Model Metrics:", metrics_merged)

# Lưu metrics của mô hình gộp
with open("merged_model_metrics_dama_2.json", "w") as f:
    json.dump(metrics_merged, f, indent=4)

Unused kwargs: ['bnb_8bit_compute_dtype', 'bnb_8bit_use_double_quant']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.
loading file vocab.json from cache at /root/.cache/huggingface/hub/models--vietgpt--dama-2-7b-chat/snapshots/efacdb053b58e26fe1de9e6b6faf0224367f78a1/vocab.json
loading file merges.txt from cache at /root/.cache/huggingface/hub/models--vietgpt--dama-2-7b-chat/snapshots/efacdb053b58e26fe1de9e6b6faf0224367f78a1/merges.txt
loading file tokenizer.json from cache at /root/.cache/huggingface/hub/models--vietgpt--dama-2-7b-chat/snapshots/efacdb053b58e26fe1de9e6b6faf0224367f78a1/tokenizer.json
loading file added_tokens.json from cache at /root/.cache/huggingface/hub/models--vietgpt--dama-2-7b-chat/snapshots/efacdb053b58e26fe1de9e6b6faf0224367f78a1/added_tokens.json
loading file special_tokens_map.json from cache at /root/.cache/huggingface/hub/models--vietgpt--dama-2-7b-chat/snapshots/efacdb053b58e26fe1de9e6b6faf0224367f78a1/s

ValueError: Some modules are dispatched on the CPU or the disk. Make sure you have enough GPU RAM to fit the quantized model. If you want to dispatch the model on the CPU or the disk while keeping these modules in 32-bit, you need to set `llm_int8_enable_fp32_cpu_offload=True` and pass a custom `device_map` to `from_pretrained`. Check https://huggingface.co/docs/transformers/main/en/main_classes/quantization#offload-between-cpu-and-gpu for more details. 

Attempting to convert .bin model on the fly to safetensors.
